# Init

In [ ]:
try:
  import google.colab
  IS_COLAB = True
except:
  IS_COLAB = False

In [ ]:
if IS_COLAB:
  !pip install tensorflow_addons
  !pip install unidecode
  !pip install transformers
  !pip install nlpaug
#  !python -m spacy download ro_core_news_lg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 13.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 73.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 KB 3.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/SemiSupervised/*.csv .

In [1]:
%%capture
import tensorflow as tf
import tensorflow_addons as tfa
import pandas as pd
import re
import spacy
# import nlpaug
import numpy as np
from transformers import AutoTokenizer, TFAutoModel
from unidecode import unidecode

2023-03-31 07:57:40.367033: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-31 07:57:40.893751: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.1/lib64:/usr/local/cuda-10.2/lib64:/usr/local/cuda/extras/CUPTI/lib64::/usr/local/cuda-10.1/lib64:/usr/local/cuda-10.2/lib64:/usr/local/cuda/extras/CUPTI/lib64:
2023-03-31 07:57:40.893811: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object 

In [2]:
gpus = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(gpus[1], 'GPU')

In [3]:
data = pd.read_csv("train_ner.csv")
data.head()

,id,text,label
0,32674,da de unde stii u mai [ORG] ca banii au fost p...,ABUSE
1,16514,m*uie [PERS] m*uie [PERS] ... m*uie\nbai kakat...,INSULT
2,32556,PT ALA CARE ARE TREABA CU [PERS]!!ESTI UNUL CA...,OTHER
3,23861,sunt bucuros ca [PERS] nu a mai venit la [ORG]...,OTHER
4,21811,[PERS] esti....PE..N\n ES..CU..LI..BI.L!!! te ...,INSULT


# Utils

In [4]:
def preprocess(x):
    s = unidecode(x)
    s = str.lower(s)
    s = re.sub(r"\[[a-z]+\]","", s)
    s = re.sub(r"\*","", s)
    s = re.sub(r"[^a-zA-Z0-9]+"," ",s)
    s = re.sub(r" +"," ",s)
    s = re.sub(r"(.)\1+",r"\1",s)

    return s

In [5]:
label_ids = {label_name:i for i, label_name in enumerate(sorted(set(data["label"])))}
label_ids

{'ABUSE': 0, 'INSULT': 1, 'OTHER': 2, 'PROFANITY': 3}

In [6]:
data["preprocessed"] = data["text"].apply(preprocess)
data.head()

,id,text,label,preprocessed
0,32674,da de unde stii u mai [ORG] ca banii au fost p...,ABUSE,da de unde sti u mai ca bani au fost pt si nu ...
1,16514,m*uie [PERS] m*uie [PERS] ... m*uie\nbai kakat...,INSULT,muie muie muie bai kakatule stai in banca ta d...
2,32556,PT ALA CARE ARE TREABA CU [PERS]!!ESTI UNUL CA...,OTHER,pt ala care are treaba cu esti unul care nu ar...
3,23861,sunt bucuros ca [PERS] nu a mai venit la [ORG]...,OTHER,sunt bucuros ca nu a mai venit la jucator de d...
4,21811,[PERS] esti....PE..N\n ES..CU..LI..BI.L!!! te ...,INSULT,esti pe n es cu li bi l te asemeni cu de la c...


In [7]:
data["class"] = data["label"].map(lambda x: label_ids[x])
data.head(5)

,id,text,label,preprocessed,class
0,32674,da de unde stii u mai [ORG] ca banii au fost p...,ABUSE,da de unde sti u mai ca bani au fost pt si nu ...,0
1,16514,m*uie [PERS] m*uie [PERS] ... m*uie\nbai kakat...,INSULT,muie muie muie bai kakatule stai in banca ta d...,1
2,32556,PT ALA CARE ARE TREABA CU [PERS]!!ESTI UNUL CA...,OTHER,pt ala care are treaba cu esti unul care nu ar...,2
3,23861,sunt bucuros ca [PERS] nu a mai venit la [ORG]...,OTHER,sunt bucuros ca nu a mai venit la jucator de d...,2
4,21811,[PERS] esti....PE..N\n ES..CU..LI..BI.L!!! te ...,INSULT,esti pe n es cu li bi l te asemeni cu de la c...,1


In [ ]:
cback = tf.keras.callbacks.EarlyStopping(
  monitor='val_f1_micro',
  min_delta=0,
  patience=2,
  mode='max',
  verbose=2,
  baseline=None,
  restore_best_weights=True
)

In [ ]:
val_data = pd.read_csv("validation_internal_ner.csv")
val_data["label"].value_counts()

OTHER        725
ABUSE        570
INSULT       435
PROFANITY    261
Name: label, dtype: int64

In [ ]:
data["label"].value_counts()

OTHER        3649
ABUSE        2768
INSULT       2242
PROFANITY    1294
Name: label, dtype: int64

# RoBERT

In [50]:
tok_robert = AutoTokenizer.from_pretrained("readerbench/RoBERT-base")
robert = TFAutoModel.from_pretrained("readerbench/RoBERT-base")

In [51]:
robert.trainable

True

In [52]:
def preprocess_robert(x):
  t = tok_robert(x,padding="max_length",max_length=96,truncation=True,return_tensors='np')
  return t["input_ids"], t["attention_mask"]

def map_func(input_ids, masks, labels):
    return {'input_ids': input_ids, 'attention_mask': masks}, labels

def prepare_ds(filename,batch_size=16):
  df = pd.read_csv(filename)
  X_id_mask = df['text'].map(preprocess).apply(preprocess_robert).apply(pd.Series)

  X_id_mask.columns = ["input_ids","attention_mask"]

  ids_array = np.squeeze(np.stack(X_id_mask.input_ids.values), axis=1)
  mask_array = np.squeeze(np.stack(X_id_mask.attention_mask.values), axis=1)
  labels = df["label"].map(lambda x: label_ids[x]).values
  res_ds = tf.data.Dataset.from_tensor_slices((ids_array, mask_array, labels)).map(map_func).shuffle(len(df)).batch(batch_size)
  return res_ds


In [53]:
def split_ssl_data(ids_array,mask_array,labels,num_classes,label_percent):
  labeled = None
  unlabeled = None

  for class_idx in range(num_classes):
    class_ids = ids_array[labels==class_idx]
    class_mask = mask_array[labels==class_idx]
    sz = int(label_percent * class_ids.shape[0])

    labels_reduced = labels[labels==class_idx][:sz]
    labeled_ids, unlabeled_ids = class_ids[:sz], class_ids[sz:]
    labeled_mask, unlabeled_mask = class_mask[:sz], class_mask[sz:]

    if not labeled:
      labeled = (labeled_ids, labeled_mask, labels_reduced)
      unlabeled = (unlabeled_ids, unlabeled_mask)
    else:
      labeled = (
          np.concatenate([labeled[0],labeled_ids]),
          np.concatenate([labeled[1],labeled_mask]),
          np.concatenate([labeled[2],labels_reduced])
      )
      unlabeled = (
          np.concatenate([unlabeled[0],unlabeled_ids]),
          np.concatenate([unlabeled[1],unlabeled_mask]),
      )

  return labeled, unlabeled

def map_func2(input_ids, masks):
  return {'input_ids': input_ids, 'attention_mask': masks}

def prepare_train_ds(filename,batch_size=16,label_percent=0.05):
  df = pd.read_csv(filename)
  df = df.sample(frac=1)
  X_id_mask = df['text'].map(preprocess).apply(preprocess_robert).apply(pd.Series)

  X_id_mask.columns = ["input_ids","attention_mask"]

  ids_array = np.squeeze(np.stack(X_id_mask.input_ids.values), axis=1)
  mask_array = np.squeeze(np.stack(X_id_mask.attention_mask.values), axis=1)
  labels = df["label"].map(lambda x: label_ids[x]).values

  labeled, unlabeled = split_ssl_data(ids_array,mask_array,labels,len(label_ids),label_percent)
  labeled_ds = tf.data.Dataset.from_tensor_slices(labeled)
  labeled_ds = labeled_ds.map(map_func).shuffle(len(labeled_ds)).batch(batch_size)
  unlabeled_ds = tf.data.Dataset.from_tensor_slices(unlabeled)
  unlabeled_ds = unlabeled_ds.map(map_func2).shuffle(len(unlabeled_ds)).batch(batch_size)
  
  return labeled_ds, unlabeled_ds

In [54]:
train_ds = prepare_train_ds("train_ner.csv")
test_ds = prepare_ds("test_ner.csv")
val_ds = prepare_ds("validation_internal_ner.csv")

In [ ]:
def get_model_bert():

  input_ids = tf.keras.Input(shape=(96,), name='input_ids', dtype='int32')
  mask = tf.keras.Input(shape=(96,), name='attention_mask', dtype='int32')

  embeddings = robert(input_ids, attention_mask=mask).last_hidden_state

  rnn_layer = tf.keras.layers.Bidirectional(
      tf.keras.layers.LSTM(
          units=10
      ),
      merge_mode="ave"
  )

  h = rnn_layer(embeddings)
  fc1 = tf.keras.layers.Dense(len(label_ids),activation='softmax')
  
  output = fc1(h)
  model = tf.keras.Model(inputs=[input_ids,mask],outputs=output)

  f1_metric_micro = tfa.metrics.F1Score(num_classes=4, threshold=0.5, average='micro', name='f1_micro')
  f1_metric_macro = tfa.metrics.F1Score(num_classes=4, threshold=0.5, average='macro', name='f1_macro')
  optim = tfa.optimizers.AdamW(weight_decay=0.001,learning_rate=0.001)
  model.compile(
      optimizer=optim,
      loss=tf.keras.losses.CategoricalCrossentropy(),
      metrics=[
          tf.keras.metrics.CategoricalAccuracy(),
          tf.keras.metrics.Precision(name='precision'),
          tf.keras.metrics.Recall(name='recall'),
          f1_metric_micro,
          f1_metric_macro
      ],
  )

  return model

In [55]:
def get_cls_head():
    inputs = tf.keras.Input(shape=(768,),dtype=tf.float32)

    fc0 = tf.keras.layers.Dense(256,activation='relu')
    drop = tf.keras.layers.Dropout(0.2)
    fc00 = tf.keras.layers.Dense(64,activation='relu')

    h = fc00(drop(fc0(inputs)))
    fc1 = tf.keras.layers.Dense(len(label_ids),activation='softmax')

    output = fc1(h)
    model = tf.keras.Model(inputs=inputs,outputs=output)

    return model


In [56]:
classifier = get_cls_head()

In [57]:
optim = tfa.optimizers.AdamW(weight_decay=0.001,learning_rate=0.001)
optim2 = tfa.optimizers.AdamW(weight_decay=0.0,learning_rate=0.00001)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
val_acc_metric = tf.keras.metrics.CategoricalAccuracy()
val_prec_metric = tf.keras.metrics.Precision(name="precision")
val_recall_metric = tf.keras.metrics.Recall(name="recall")
f1_metric_micro = tfa.metrics.F1Score(num_classes=4, threshold=0.5, average='micro', name='f1_micro')
f1_metric_macro = tfa.metrics.F1Score(num_classes=4, threshold=0.5, average='macro', name='f1_macro')

In [58]:
@tf.function
def train_step(x,y):

    with tf.GradientTape(persistent=True) as tape:
        embeds = robert(input_ids=x["input_ids"],attention_mask=x["attention_mask"],training=True).pooler_output
        preds = classifier(embeds,training=True)
        loss = loss_fn(y, preds)
    
    grads = tape.gradient(loss, classifier.trainable_variables)
    optim.apply_gradients(zip(grads, classifier.trainable_variables))
    grads = tape.gradient(loss, robert.trainable_variables)
    optim2.apply_gradients(zip(grads, robert.trainable_variables))

    return loss

@tf.function
def test_step(x, y):
    emb = robert(input_ids=x["input_ids"],attention_mask=x["attention_mask"], training=False).pooler_output
    wpred = classifier(emb,training=False)
    true_hot = tf.one_hot(y, 4)
    val_acc_metric.update_state(true_hot, wpred)
    val_prec_metric.update_state(true_hot, wpred)
    val_recall_metric.update_state(true_hot, wpred)
    f1_metric_micro.update_state(true_hot, wpred)
    f1_metric_macro.update_state(true_hot, wpred)
    

In [59]:
import time

In [60]:
max_val_acc = 0.0
best_weights = None
best_robert = None
for epoch in range(15):
  print(f"Epoch {epoch}")
  start_time = time.time()
  step = 0
  for x,y in train_ds[0]:
    loss = train_step(x,y)

    if step % 50 == 0:
      print(
        "Training loss (for one batch) at step %d: %.4f"
        % (step, float(loss))
      )

    step += 1
    

  for x_batch_val, y_batch_val in val_ds:
      test_step(x_batch_val, y_batch_val)

  acc = float(val_acc_metric.result())
  prec = float(val_prec_metric.result())
  recall = float(val_recall_metric.result())
  micro = float(f1_metric_micro.result())
  macro = float(f1_metric_macro.result())

  val_acc_metric.reset_states()
  val_prec_metric.reset_states()
  val_recall_metric.reset_states()
  f1_metric_micro.reset_states()
  f1_metric_macro.reset_states()

  if acc > max_val_acc:
    max_val_acc = acc
    best_weights = classifier.get_weights()
    best_robert = robert.get_weights()

  print(f"Validation acc: {acc:.3f} precision: {prec:.3f} recall: {recall:.3f} f1_micro: {micro:.3f} f1_macro: {macro:.3f}")
  print("Time taken: %.2fs" % (time.time() - start_time))

classifier.set_weights(best_weights)
robert.set_weights(best_robert)

Epoch 0
Training loss (for one batch) at step 0: 1.4480
Validation acc: 0.519 precision: 0.614 recall: 0.355 f1_micro: 0.450 f1_macro: 0.285
Time taken: 21.99s
Epoch 1
Training loss (for one batch) at step 0: 0.8080
Validation acc: 0.544 precision: 0.643 recall: 0.402 f1_micro: 0.495 f1_macro: 0.306
Time taken: 12.25s
Epoch 2
Training loss (for one batch) at step 0: 0.7917
Validation acc: 0.543 precision: 0.597 recall: 0.474 f1_micro: 0.528 f1_macro: 0.380
Time taken: 12.15s
Epoch 3
Training loss (for one batch) at step 0: 0.5316
Validation acc: 0.525 precision: 0.553 recall: 0.476 f1_micro: 0.512 f1_macro: 0.409
Time taken: 12.19s
Epoch 4
Training loss (for one batch) at step 0: 0.2786
Validation acc: 0.566 precision: 0.586 recall: 0.536 f1_micro: 0.560 f1_macro: 0.493
Time taken: 12.37s
Epoch 5
Training loss (for one batch) at step 0: 0.1591
Validation acc: 0.567 precision: 0.580 recall: 0.548 f1_micro: 0.563 f1_macro: 0.504
Time taken: 12.40s
Epoch 6
Training loss (for one batch) at

In [61]:
for x_batch_val, y_batch_val in test_ds:
    test_step(x_batch_val, y_batch_val)
acc = float(val_acc_metric.result())
prec = float(val_prec_metric.result())
recall = float(val_recall_metric.result())
micro = float(f1_metric_micro.result())
macro = float(f1_metric_macro.result())

val_acc_metric.reset_states()
val_prec_metric.reset_states()
val_recall_metric.reset_states()
f1_metric_micro.reset_states()
f1_metric_macro.reset_states()
print(f"Test acc: {acc:.3f} precision: {prec:.3f} recall: {recall:.3f} f1_micro: {micro:.3f} f1_macro: {macro:.3f}")

Test acc: 0.591 precision: 0.595 recall: 0.585 f1_micro: 0.590 f1_macro: 0.552


In [ ]:
robert.trainable = False
model = get_model_bert()
history = model.fit(x=train_ds, epochs=20, validation_data=val_ds, callbacks=[cback])

Epoch 1/20
156/156 [==============================] - 102s 547ms/step - loss: 1.1277 - categorical_accuracy: 0.5075 - precision: 0.6777 - recall: 0.2891 - f1_micro: 0.4053 - f1_macro: 0.2613 - val_loss: 0.9940 - val_categorical_accuracy: 0.5781 - val_precision: 0.7026 - val_recall: 0.4023 - val_f1_micro: 0.5117 - val_f1_macro: 0.3322
Epoch 2/20
156/156 [==============================] - 82s 522ms/step - loss: 0.9826 - categorical_accuracy: 0.5759 - precision: 0.7044 - recall: 0.4023 - f1_micro: 0.5121 - f1_macro: 0.3484 - val_loss: 0.9615 - val_categorical_accuracy: 0.5691 - val_precision: 0.6811 - val_recall: 0.4194 - val_f1_micro: 0.5191 - val_f1_macro: 0.3512
Epoch 3/20
156/156 [==============================] - 81s 520ms/step - loss: 0.9285 - categorical_accuracy: 0.6093 - precision: 0.7162 - recall: 0.4456 - f1_micro: 0.5494 - f1_macro: 0.4245 - val_loss: 0.8659 - val_categorical_accuracy: 0.6489 - val_precision: 0.7530 - val_recall: 0.5008 - val_f1_micro: 0.6015 - val_f1_macro: 0

In [ ]:
model.evaluate(test_ds)

39/39 [==============================] - 16s 402ms/step - loss: 0.7312 - categorical_accuracy: 0.7079 - precision: 0.7740 - recall: 0.6348 - f1_micro: 0.6975 - f1_macro: 0.6694


[0.7311965227127075,
 0.7078651785850525,
 0.7739726305007935,
 0.6348314881324768,
 0.6975308656692505,
 0.669391393661499]

# Embedding Layer

In [ ]:
MAX_FEATURES = 20000
SEQUENCE_LENGTH = 50
BATCH_SIZE = 64
EMBEDDING_DIM = 10
EPOCHS = 10

In [ ]:
def prepare_dataset(filename):
    df = pd.read_csv(filename)
    df["preprocessed"] = df["text"].map(preprocess)
    labels = df["label"].map(lambda x: label_ids[x]).apply(lambda x: tf.keras.utils.to_categorical(int(x), len(label_ids))).values.tolist()
    res_ds = tf.data.Dataset.from_tensor_slices((df["preprocessed"],labels)).shuffle(len(df)).batch(BATCH_SIZE)
    return res_ds

In [ ]:
train_ds = prepare_dataset("train_ner.csv")
test_ds = prepare_dataset("test_ner.csv")
val_ds = prepare_dataset("validation_internal_ner.csv")

NameError: ignored

In [ ]:
texts = train_ds.map(lambda x,y:x)
vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=MAX_FEATURES,
    output_mode="int",
    output_sequence_length=SEQUENCE_LENGTH,
    standardize=None,
    split="whitespace",
)
vectorizer.adapt(texts)

In [ ]:
#@title get_model
def get_model():

  input = tf.keras.Input(shape=(),dtype=tf.string)
  tokens = vectorizer(input)
  embed_layer = tf.keras.layers.Embedding(
      input_dim=MAX_FEATURES,
      output_dim=EMBEDDING_DIM,
      mask_zero=True
  )
  embeds = embed_layer(tokens)
  rnn_layer = tf.keras.layers.Bidirectional(
      tf.keras.layers.LSTM(
          units=10
      ),
      merge_mode="ave"
  )

  h = rnn_layer(embeds)

  fc2 = tf.keras.layers.Dense(len(label_ids),activation='softmax')

  output = fc2(h)

  model = tf.keras.Model(inputs=input,outputs=output)

  f1_metric_micro = tfa.metrics.F1Score(num_classes=4, threshold=0.5, average='micro', name='f1_micro')
  f1_metric_macro = tfa.metrics.F1Score(num_classes=4, threshold=0.5, average='macro', name='f1_macro')
  optim = tfa.optimizers.AdamW(weight_decay=0.001,learning_rate=0.001)
  model.compile(
      optimizer=optim,
      loss=tf.keras.losses.CategoricalCrossentropy(),
      metrics=[
          tf.keras.metrics.CategoricalAccuracy(),
          tf.keras.metrics.Precision(name='precison'),
          tf.keras.metrics.Recall(name='recall'),
          f1_metric_micro,
          f1_metric_macro
      ],
  )

  return model

In [ ]:
r = []
for _ in range(7):
  model = get_model()
  model.fit(x=train_ds, epochs=4, validation_data=val_ds, callbacks=[cback],verbose=0)
  eval_res = model.evaluate(test_ds)
  print(eval_res)
  r.append(eval_res)
print(np.mean(np.array(r),axis=0))

39/39 [==============================] - 0s 9ms/step - loss: 0.8728 - categorical_accuracy: 0.6894 - precison: 0.7458 - recall: 0.6075 - f1_micro: 0.6696 - f1_macro: 0.6339
[0.8727979063987732, 0.6894060969352722, 0.7458128333091736, 0.6075441241264343, 0.6696152091026306, 0.6338611841201782]
39/39 [==============================] - 0s 8ms/step - loss: 0.8865 - categorical_accuracy: 0.6685 - precison: 0.7447 - recall: 0.5618 - f1_micro: 0.6404 - f1_macro: 0.5764
[0.8864669799804688, 0.6685393452644348, 0.7446808218955994, 0.5617977380752563, 0.6404391527175903, 0.5763561129570007]
39/39 [==============================] - 0s 8ms/step - loss: 0.9437 - categorical_accuracy: 0.6180 - precison: 0.7083 - recall: 0.4755 - f1_micro: 0.5690 - f1_macro: 0.4551
[0.9436929821968079, 0.617977499961853, 0.7083083987236023, 0.47552168369293213, 0.5690276622772217, 0.45505809783935547]
39/39 [==============================] - 0s 8ms/step - loss: 0.9228 - categorical_accuracy: 0.6413 - precison: 0.7419

# XLM-R multilingual

In [ ]:
tok_xlm = AutoTokenizer.from_pretrained("sentence-transformers/paraphrase-xlm-r-multilingual-v1")
xlmr = TFAutoModel.from_pretrained("sentence-transformers/paraphrase-xlm-r-multilingual-v1")

In [ ]:
def preprocess_xlm(x):
  t = tok_xlm(x,padding="max_length",max_length=70,truncation=True,return_tensors='np')
  return t["input_ids"], t["attention_mask"]

def map_func(input_ids, masks, labels):
    return {'input_ids': input_ids, 'attention_mask': masks}, labels

def prepare_ds2(filename,batch_size=64):
  df = pd.read_csv(filename)
  X_id_mask = df['text'].map(preprocess).apply(preprocess_xlm).apply(pd.Series)

  X_id_mask.columns = ["input_ids","attention_mask"]

  ids_array = np.squeeze(np.stack(X_id_mask.input_ids.values), axis=1)
  mask_array = np.squeeze(np.stack(X_id_mask.attention_mask.values), axis=1)
  labels = df["label"].map(lambda x: label_ids[x]).apply(lambda x: tf.keras.utils.to_categorical(int(x), len(label_ids))).values.tolist()
  res_ds = tf.data.Dataset.from_tensor_slices((ids_array, mask_array, labels)).map(map_func).shuffle(len(df)).batch(batch_size)
  return res_ds

In [ ]:
train_ds = prepare_ds2("train_ner.csv")
test_ds = prepare_ds2("test_ner.csv")
val_ds = prepare_ds2("validation_internal_ner.csv")

In [ ]:
#@title get_model_xlm
def get_model_xlm():

  input_ids = tf.keras.Input(shape=(70,), name='input_ids', dtype='int32')
  mask = tf.keras.Input(shape=(70,), name='attention_mask', dtype='int32')

  embeddings = xlmr(input_ids, attention_mask=mask).pooler_output

  fc1 = tf.keras.layers.Dense(128,activation='relu')
  fc2 = tf.keras.layers.Dense(len(label_ids),activation='softmax')

  output = fc2(fc1(embeddings))
  model = tf.keras.Model(inputs=[input_ids,mask],outputs=output)

  f1_metric_micro = tfa.metrics.F1Score(num_classes=4, threshold=0.5, average='micro', name='f1_micro')
  f1_metric_macro = tfa.metrics.F1Score(num_classes=4, threshold=0.5, average='macro', name='f1_macro')
  optim = tfa.optimizers.AdamW(weight_decay=0.001,learning_rate=0.001)
  model.compile(
      optimizer=optim,
      loss=tf.keras.losses.CategoricalCrossentropy(),
      metrics=[
          tf.keras.metrics.CategoricalAccuracy(),
          tf.keras.metrics.Precision(name='precision'),
          tf.keras.metrics.Recall(name='recall'),
          f1_metric_micro,
          f1_metric_macro
      ],
  )

  return model

In [ ]:
xlmr.trainable = False
model = get_model_xlm()
history = model.fit(x=train_ds, epochs=20, validation_data=val_ds, callbacks=[cback])
print(history.history)

Epoch 1/20
78/78 [==============================] - 76s 802ms/step - loss: 1.2453 - categorical_accuracy: 0.4576 - precision: 0.6809 - recall: 0.0832 - f1_micro: 0.1483 - f1_macro: 0.0950 - val_loss: 1.1626 - val_categorical_accuracy: 0.5063 - val_precision: 0.6738 - val_recall: 0.2220 - val_f1_micro: 0.3340 - val_f1_macro: 0.2130
Epoch 2/20
78/78 [==============================] - 58s 740ms/step - loss: 1.1652 - categorical_accuracy: 0.4985 - precision: 0.6592 - recall: 0.2299 - f1_micro: 0.3409 - f1_macro: 0.2190 - val_loss: 1.1297 - val_categorical_accuracy: 0.5193 - val_precision: 0.7039 - val_recall: 0.2341 - val_f1_micro: 0.3513 - val_f1_macro: 0.2383
Epoch 3/20
78/78 [==============================] - 58s 747ms/step - loss: 1.1359 - categorical_accuracy: 0.5109 - precision: 0.6661 - recall: 0.2680 - f1_micro: 0.3822 - f1_macro: 0.2484 - val_loss: 1.1011 - val_categorical_accuracy: 0.5299 - val_precision: 0.6745 - val_recall: 0.3164 - val_f1_micro: 0.4308 - val_f1_macro: 0.2727
E

In [ ]:
model.evaluate(test_ds)

20/20 [==============================] - 11s 541ms/step - loss: 1.0964 - categorical_accuracy: 0.5157 - precision: 0.6797 - recall: 0.3347 - f1_micro: 0.4485 - f1_macro: 0.2864


[1.0964308977127075,
 0.5156500935554504,
 0.6797065734863281,
 0.3346709609031677,
 0.4485076665878296,
 0.28639763593673706]

# Experiments and EDA

In [ ]:
X_id_mask = data['text'].apply(preprocess_robert).apply(pd.Series)

X_id_mask.columns = ["input_ids","attention_mask"]

ids_array = np.squeeze(np.stack(X_id_mask.input_ids.values), axis=1)
mask_array = np.squeeze(np.stack(X_id_mask.attention_mask.values), axis=1)

In [ ]:
data.head()

,id,text,label,preprocessed,class
0,32674,da de unde stii u mai [ORG] ca banii au fost p...,ABUSE,da de unde sti u mai ca bani au fost pt si nu ...,0
1,16514,m*uie [PERS] m*uie [PERS] ... m*uie\nbai kakat...,INSULT,muie muie muie bai kakatule stai in banca ta d...,1
2,32556,PT ALA CARE ARE TREABA CU [PERS]!!ESTI UNUL CA...,OTHER,pt ala care are treaba cu esti unul care nu ar...,2
3,23861,sunt bucuros ca [PERS] nu a mai venit la [ORG]...,OTHER,sunt bucuros ca nu a mai venit la jucator de d...,2
4,21811,[PERS] esti....PE..N\n ES..CU..LI..BI.L!!! te ...,INSULT,esti pe n es cu li bi l te asemeni cu de la c...,1


In [ ]:
n = ids_array.shape[0]

embeds = -1
for i in range(0,n,100):
    print(i)
    end = min(i+100,n)
    res = tf.reduce_mean(robert(input_ids=ids_array[i:end,:],attention_mask=mask_array[i:end,:]).last_hidden_state,axis=1)
    if i==0:
      embeds = res
    else:
      embeds = tf.concat([embeds, res], axis=0)

embeds.shape

In [ ]:
embeds.numpy()

array([[ 0.20765497, -0.38347745, -0.20120968, ...,  0.0165822 ,
         0.21433175, -0.2159521 ],
       [-0.02741106, -0.44976708, -0.4155186 , ...,  0.07389837,
         0.22652103, -0.18264015],
       [ 0.16284953, -0.3458496 , -0.2576867 , ...,  0.39777884,
         0.34855923, -0.28040662],
       ...,
       [ 0.0502818 , -0.42666745, -0.33625123, ..., -0.04228253,
         0.04308325, -0.560236  ],
       [ 0.17077483, -0.7920572 , -0.11984698, ...,  0.03649964,
         0.11620215, -0.16182849],
       [ 0.08397353, -0.02763316,  0.13787225, ...,  0.402563  ,
         0.40377474, -0.32798257]], dtype=float32)

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
tsne = TSNE(n_components=2, learning_rate='auto').fit_transform(embeds.numpy())

/usr/local/lib/python3.8/dist-packages/sklearn/manifold/_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(


In [ ]:
tsne2 = TSNE(n_components=3, init="pca", learning_rate='auto').fit_transform(embeds.numpy())

/usr/local/lib/python3.8/dist-packages/sklearn/manifold/_t_sne.py:982: FutureWarning:

The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.



In [ ]:
tsne.shape

(9953, 2)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=tsne[:,0],y=tsne[:,1],mode="markers",marker_color=data["class"]))
fig.show()

In [ ]:
fig2 = go.Figure()
fig2.add_trace(go.Scatter3d(x=tsne2[:,0],y=tsne2[:,1],z=tsne2[:,2],mode="markers",marker_color=data["class"]))
fig2.show()

In [ ]:
l = [elem.split(" ") for elem in data["preprocessed"].tolist()]
s = set()
for elem in l:
    s = s|set(elem)
print(len(s))

27240


In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Histogram(x=x))
fig.show()

In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 KB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 KB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 10.9 MB/s eta 0:00:00


In [ ]:
import optuna

In [ ]:
def get_model2(trial):


  model = tf.keras.Sequential()
  model.add(vectorizer)

  edim = trial.suggest_int("embedding_dim",10,200)
  embed_layer = tf.keras.layers.Embedding(
      input_dim=MAX_FEATURES,
      output_dim=edim,
      mask_zero=True
  )

  model.add(embed_layer)
  
  
  num_rnn = trial.suggest_int("num_lstm",0,2)
  for i in range(num_rnn):
    num_hidden = trial.suggest_int(f"num_units_lstm{i}",10,50)
    model.add(tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(
          units=num_hidden,
          return_sequences=True
        ),
        merge_mode="ave"
    ))
  model.add(tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(
          units=num_hidden
        ),
        merge_mode="ave"
  ))
  
  num_dense = trial.suggest_int("num_dense",0,3)
  for i in range(num_dense):
    num_hidden = trial.suggest_int(f"num_units_dense{i}",4,10)
    model.add(tf.keras.layers.Dense(num_hidden,activation='relu'))

  model.add(
      tf.keras.layers.Dense(len(label_ids),activation='softmax')
  )
  
  f1_metric_micro = tfa.metrics.F1Score(num_classes=4, threshold=0.5, average='micro', name='f1_micro')
  f1_metric_macro = tfa.metrics.F1Score(num_classes=4, threshold=0.5, average='macro', name='f1_macro')
  optim = tfa.optimizers.AdamW(weight_decay=0.001,learning_rate=0.001)
  model.compile(
      optimizer=optim,
      loss=tf.keras.losses.CategoricalCrossentropy(),
      metrics=[
          tf.keras.metrics.CategoricalAccuracy(),
          tf.keras.metrics.Precision(name='precison'),
          tf.keras.metrics.Recall(name='recall'),
          f1_metric_micro,
          f1_metric_macro
      ],
  )

  return model

In [ ]:
def objective(trial):
    model = get_model2(trial)
    hist = model.fit(x=train_ds, epochs=4, validation_data=test_ds,verbose=0)
    return hist.history["val_categorical_accuracy"][-1]

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

print("Number of finished trials: ", len(study.trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2023-02-05 15:44:43,398] A new study created in memory with name: no-name-4dec5e9e-c42a-4e0d-a947-e20fff82f6df
[I 2023-02-05 15:45:57,952] Trial 0 finished with value: 0.5321027040481567 and parameters: {'embedding_dim': 58, 'num_lstm': 2, 'num_units_lstm0': 11, 'num_units_lstm1': 19, 'num_dense': 1, 'num_units_dense0': 9}. Best is trial 0 with value: 0.5321027040481567.
[I 2023-02-05 15:46:54,162] Trial 1 finished with value: 0.5349116921424866 and parameters: {'embedding_dim': 124, 'num_lstm': 1, 'num_units_lstm0': 38, 'num_dense': 2, 'num_units_dense0': 5, 'num_units_dense1': 6}. Best is trial 1 with value: 0.5349116921424866.
[I 2023-02-05 15:48:12,989] Trial 2 finished with value: 0.5276886224746704 and parameters: {'embedding_dim': 101, 'num_lstm': 2, 'num_units_lstm0': 18, 'num_units_lstm1': 46, 'num_dense': 2, 'num_units_dense0': 5, 'num_units_dense1': 9}. Best is trial 1 with value: 0.5349116921424866.
[I 2023-02-05 15:49:32,186] Trial 3 finished with value: 0.4052969515323

KeyboardInterrupt: ignored

In [ ]:
m/min(m)

array([2.81993818, 2.13910355, 1.73261206, 1.        ])

In [ ]:
test_data = pd.read_csv("test_ner.csv")
m2 = test_data["label"].value_counts().values
m2 / min(m2)

array([2.71299094, 2.06646526, 1.74924471, 1.        ])

In [ ]:
lengths = np.array([len(elem.split(" ")) for elem in data["preprocessed"]])

In [ ]:
print(np.mean(lengths), np.std(lengths))

17.273384909072643 10.580427950116055


In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Histogram(x=lengths))
fig.show()

In [ ]:
import nlpaug.augmenter.word as naw

In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
aug = naw.BackTranslationAug(from_model_name="Helsinki-NLP/opus-mt-ro-fr",to_model_name="Helsinki-NLP/opus-mt-fr-ro",device="cuda")

/usr/local/lib/python3.8/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
aug.augment("unde a fost STEAUA in ultimii 3 ani???...semifinala UEFA si grupele UCL....asa k STEAUA e STEAUA...M**E LA TOTI UNGURII SI TZIGANII!!! RESPECT 4EVER MARESALU ANTONESCU! ")

['Unde a fost steaua în ultimii trei ani?....semifinală a UEFA şi a grupurilor UCL... dacă k steaua este steaua... M*Toţi maghiarii şi Thsiganii!!!!']

In [ ]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
aug = naw.synonym.SynonymAug(lang="ron")

In [ ]:
aug.augment("competitia este dificila.")

['competitia este dificila.']

In [ ]:
from nltk.corpus import wordnet as wn
from nltk.stem.snowball import RomanianStemmer

In [ ]:
stemmer = RomanianStemmer()

In [ ]:
[[x for x in elem.lemma_names("ron")] for elem in wn.synsets("competiție",lang="ron")]

[['competiție', 'concurs', 'întrecere']]

In [ ]:
wn.langs()

dict_keys(['eng', 'als', 'arb', 'bul', 'cmn', 'dan', 'ell', 'fin', 'fra', 'heb', 'hrv', 'isl', 'ita', 'ita_iwn', 'jpn', 'cat', 'eus', 'glg', 'spa', 'ind', 'zsm', 'nld', 'nno', 'nob', 'pol', 'por', 'ron', 'lit', 'slk', 'slv', 'swe', 'tha'])